In [3]:
import time
from laser import LaserControl

# test usblib--------------------------------
import usb
las = usb.core.find(find_all=True)
# las = usb.core.find(find_all=True, idVendor=0x0403, idProduct=0x90D9)
laslist = []
for item in las:
    laslist.append(item)
print(laslist)
# command = '?HID'
# ttt = laslist[0].write(0x02, (command+'\0').encode(), 100)
# rep = laslist[0].read(0x81, 30, 1000)
# usb.util.dispose_resources(laslist[0]) # Important!! 
# -------------------------------------
laser = LaserControl()
laser.open()
# laser.close()

# before all the tests: close the oxxius software

[<DEVICE ID 0403:90d9 on Bus 000 Address 001>]


In [4]:
# test laser on/off ------------------------------
laser.laser_on()
assert laser.is_laser_on()
laser.laser_off()
assert not laser.is_laser_on()
assert laser.get_status() == "Standby"

In [5]:
# test constant mode ---------------------------
laser.set_analog_control_mode("power")
assert laser.get_analog_control_mode() == "power"
laser.set_analog_control_mode("current")
assert laser.get_analog_control_mode() != "power"

In [6]:
# test analog modulation -----------------------
laser.set_analog_modulation("iNt")
assert laser.get_analog_modulation() == "int"
laser.set_analog_modulation("Ext")
assert laser.get_analog_modulation() != "int"

In [8]:
# test diode current ----------------------------------
current_percent = 20.75
laser.set_analog_control_mode("current")
laser.set_diode_current(current_percent, save_memory=False)
laser.laser_on()
time.sleep(10) # allow laser current to settle
cp_read = laser.get_diode_current()/laser.get_max_laser_current()
laser.laser_off()
assert round(current_percent, -1) == round(cp_read*100, -1)

AssertionError: 

In [9]:
# test CDRH state ---------------------------------
laser.set_CDRH_state(True)
assert laser.get_CDRH_state()
laser.set_CDRH_state(False)
assert laser.get_CDRH_state() == False

In [10]:
# test modulation type cw/modulated -------------------------
laser.set_modulation_state("CW")
assert laser.get_modulation_state() == "cw"
laser.set_modulation_state("Pulsed")
assert laser.get_modulation_state() == "modulated"

In [11]:
# test laser power setting ----------------------------
power = 32.75 # [mW]
laser.set_analog_control_mode("power")
laser.set_laser_power(power, save_memory=True)
laser.laser_on()
time.sleep(10) # allow laser current to settle
power_read = laser.get_laser_power()
laser.laser_off()
assert round(power) == round(power_read)

Failed to set command 'P=33'
Response from Device :'P=33.0'


AssertionError: 

In [12]:
# test alarm reset --------------------------------
laser.laser_off()
laser.reset_alarm()
time.sleep(10) # wait for reset
assert laser.get_status() == "Laser ON"
laser.laser_off()
assert laser.get_status() == "Standby"

Failed to set command 'RST'
Response from Device :'RST OK'


AssertionError: 

In [13]:
# test TEC states -------------------------------
laser.set_TEC_enable(enable=False)
assert laser.get_TEC_state() == False
assert laser.get_status() == "Sleep"
laser.set_TEC_enable()
assert laser.get_TEC_state()

In [14]:
# test TTL digital modulation ---------------------
laser.set_analog_control_mode("power")
laser.set_digital_modulation("ExT")
assert laser.get_digital_modulation() != "ext"
laser.set_analog_control_mode("current")
laser.set_digital_modulation("ExT")
assert laser.get_digital_modulation() == "ext"
laser.set_digital_modulation("Int")
assert laser.get_digital_modulation() == 'int'

Failed to set command 'TTL=1'
Response from Device :'TTL=0'


In [15]:
laser.close() # IMPORTANT to close the connection!!

In [ ]:
adictlist = [{"value":523, "disable":True}, {"value":32432, "disable":True}, {"value":5325, "disable":True}]
